In [0]:
%pip install seaborn
%pip install xgboost
%pip install lightgbm
%pip install imbalanced-learn
%pip install feature-engine

In [0]:
%pip install pandas==1.5.3
# --- Bibliotecas para Manipulação de Dados ---
import pandas as pd
import numpy as np

# --- Bibliotecas para Visualização ---
import seaborn as sns
import matplotlib.pyplot as plt

# --- Ferramentas do Scikit-learn para Pré-processamento e Pipelines ---
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline # O pipeline padrão do Scikit-learn

# --- NOVAS: Bibliotecas para Modelagem e Balanceamento ---

# Para balanceamento de classes com SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline # Usamos um alias para não confundir com o pipeline do sklearn
from imblearn.over_sampling import SMOTE

# Para o modelo RandomForest (que já usamos)
from sklearn.ensemble import RandomForestClassifier

# Para o modelo LightGBM
import lightgbm as lgb

# Para o modelo XGBoost
import xgboost as xgb


# --- Ferramentas para Avaliação do Modelo ---
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve

# --- Configurações Adicionais (Opcional) ---
# Para ignorar avisos que não são críticos
import warnings
warnings.filterwarnings('ignore')

# Para ver todas as colunas de um DataFrame
pd.set_option('display.max_columns', None)

print("Bibliotecas importadas com sucesso!")



In [0]:
df = pd.read_csv('cs-training.csv')
df_2 = pd.read_csv('cs-test.csv')

In [0]:
df_train = df.copy()

df_train['BaLim_ratio'] = df_train['RevolvingUtilizationOfUnsecuredLines'].clip(upper=1.2)

df_train['age_2'] = np.where(df_train['age'] == 0, 20, df_train['age'])

income_capped = df_train['MonthlyIncome'].clip(upper=23300)
# Depois, aplicamos a transformação de log
df_train['Income_log'] = np.log(income_capped + 10)
        
df_train['Income_bool'] = np.where(df_train['MonthlyIncome'] > 0, 1, 0)

df_train['Dep'] = df_train['NumberOfDependents'].clip(upper=2)

df_train['D_Ratio'] = np.where(df_train['DebtRatio'] >= 1200, 6, np.where(df_train['DebtRatio'] > 4, 5, df_train['DebtRatio']))
        
df_train['OCLAL'] = np.log1p(df_train['NumberOfOpenCreditLinesAndLoans'])

df_train['REOL'] = df_train['NumberRealEstateLoansOrLines'].clip(upper=5)

df_train['Num_30-59'] = df_train['NumberOfTime30-59DaysPastDueNotWorse'].clip(upper=3)

df_train['Num_60-89'] = df_train['NumberOfTime60-89DaysPastDueNotWorse'].clip(upper=2)

df_train['Num_90'] = df_train['NumberOfTimes90DaysLate'].clip(upper=2)

df_train['Income_log_sq'] = df_train['Income_log']**2
df_train['Income_log_cub'] = df_train['Income_log']**3
df_train['D_Ratio_sq'] = df_train['D_Ratio']**2 

df_treino_final = df_train.copy()


In [0]:
# Novas features a partir das existentes

df_teste = df_2.copy()

df_teste['BaLim_ratio'] = df_teste['RevolvingUtilizationOfUnsecuredLines'].clip(upper=1.2)

df_teste['age_2'] = np.where(df_teste['age'] == 0, 20, df_teste['age'])

income_capped = df_teste['MonthlyIncome'].clip(upper=23300)
# Depois, aplicamos a transformação de log
df_teste['Income_log'] = np.log(income_capped + 10)
        
df_teste['Income_bool'] = np.where(df_teste['MonthlyIncome'] > 0, 1, 0)

df_teste['Dep'] = df_teste['NumberOfDependents'].clip(upper=2)

df_teste['D_Ratio'] = np.where(df_teste['DebtRatio'] >= 1200, 6, np.where(df_teste['DebtRatio'] > 4, 5, df_teste['DebtRatio']))
        
df_teste['OCLAL'] = np.log1p(df_teste['NumberOfOpenCreditLinesAndLoans'])

df_teste['REOL'] = df_teste['NumberRealEstateLoansOrLines'].clip(upper=5)

df_teste['Num_30-59'] = df_teste['NumberOfTime30-59DaysPastDueNotWorse'].clip(upper=3)

df_teste['Num_60-89'] = df_teste['NumberOfTime60-89DaysPastDueNotWorse'].clip(upper=2)

df_teste['Num_90'] = df_teste['NumberOfTimes90DaysLate'].clip(upper=2)

df_teste['Income_log_sq'] = df_teste['Income_log']**2
df_teste['Income_log_cub'] = df_teste['Income_log']**3
df_teste['D_Ratio_sq'] = df_teste['D_Ratio']**2 


df_teste_final = df_teste.copy()

In [0]:
y = df_train['SeriousDlqin2yrs']
X = df_train.drop(['SeriousDlqin2yrs','age','DebtRatio','MonthlyIncome','Unnamed: 0', 'NumberOfDependents','RevolvingUtilizationOfUnsecuredLines','NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse','NumberOfTimes90DaysLate'
], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Formato dos dados de treino:", y_train.mean())
print("Formato dos dados de teste:", y_test.mean())

#X_train_eng = X_train.copy()
#X_test_eng = X_test.copy()
X_train.columns

X_train.isnull().sum()

In [0]:
model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1,
    
)

# Pipeline

pipeline_final = ImbPipeline([
    #('preprocessamento', preprocessor),
    #('sampler', SMOTE(random_state=42, sampling_strategy=1)),
    ('classificador', model)
])


# Parametros
# Grid para RandomForest

param_grid_rf = {
    'classificador': [RandomForestClassifier(random_state=42, n_jobs=-1)], # Define o modelo a ser usado
    'classificador__n_estimators': [200, 400],
    'classificador__max_depth': [5, 10, 15],
    'classificador__min_samples_leaf': [10, 20]
}

# Grid para XGBoost
param_grid_xgb = {
    # Tente aumentar o peso da classe minoritária ainda mais
    'classificador__learning_rate': [0.03], # 'eta' do R
    'classificador__max_depth': [4],
    'classificador__n_estimators': [1000],   # 'nrounds' do R
    'classificador__subsample': [0.5],
    'classificador__colsample_bytree': [0.5]
}

# Grid para LightGBM
param_grid_lgbm = {
    'classificador': [lgb.LGBMClassifier(objective='binary', metric='auc', random_state=42, n_jobs=-1)],
    'classificador__n_estimators': [500, 1000],
    'classificador__learning_rate': [0.03],
    'classificador__num_leaves': [20, 31, 40]
}

#Lista 
param_grid_completo = [param_grid_rf, param_grid_xgb, param_grid_lgbm]


grid_search = GridSearchCV(
    estimator=model, # <-- Usa o pipeline_final que acabamos de criar
    param_grid=param_grid_completo,
    scoring='roc_auc',
    cv=5,
    verbose=2,
    n_jobs=1
)



grid_search.fit(X_train, y_train)

print("Pré-processamento concluído com sucesso!")
#print("Formato dos dados de treino processados:", X_train_transf.shape)

In [0]:
# --- 5. Analise os Resultados ---

print("\nBusca concluída!")
print("\nMelhor score AUC encontrado na validação cruzada:")
print(f"{grid_search.best_score_:.4f}")

print("\nMelhor combinação de modelo e parâmetros encontrada:")
# O .best_params_ agora te dirá qual 'classificador' foi o melhor!
print(grid_search.best_params_)

# Guarde o melhor pipeline completo encontrado
melhor_modelo = grid_search.best_estimator_


# --- 6. Avalie o Campeão no seu Conjunto de Teste Local ---
y_pred = melhor_modelo.predict(X_test)
y_pred_proba = melhor_modelo.predict_proba(X_test)[:, 1]

print("\n" + "="*40)
print("AVALIAÇÃO DO MELHOR MODELO ENCONTRADO")
print("="*40)
print(classification_report(y_test, y_pred))
print(f"\nAUC Score no teste: {roc_auc_score(y_test, y_pred_proba):.4f}")

In [0]:
# Treinando com 100% do dataset
y_treino_total = df_treino_final['SeriousDlqin2yrs']
X_treino_total = df_treino_final.drop(['SeriousDlqin2yrs','age','DebtRatio','MonthlyIncome','Unnamed: 0', 'NumberOfDependents','RevolvingUtilizationOfUnsecuredLines','NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse','NumberOfTimes90DaysLate'
], axis=1)

# Definindo o conjunto teste com 100% dos dados.

X_teste_total = df_teste_final.drop(['SeriousDlqin2yrs','age','DebtRatio','MonthlyIncome','Unnamed: 0', 'NumberOfDependents','RevolvingUtilizationOfUnsecuredLines','NumberOfOpenCreditLinesAndLoans', 'NumberRealEstateLoansOrLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'NumberOfTime60-89DaysPastDueNotWorse','NumberOfTimes90DaysLate'
], axis=1)


#grid_search.fit(X_treino_total, y_treino_total)
#previsoes_finais_proba = grid_search.predict_proba(X_teste_total)[:, 1]

previsoes_finais_proba = melhor_modelo.predict_proba(X_teste_total)[:, 1]

#previsoes_finais_proba = grid_search.predict_proba(X_teste_total)[:, 1]

df_submissao = pd.DataFrame({
    'Id': range(1, len(previsoes_finais_proba) + 1),
    'Probability': previsoes_finais_proba
})

df_submissao['Probability'] = previsoes_finais_proba

print("\n--- ESTATÍSTICAS DAS PROBABILIDADES PREVISTAS ---")
print(df_submissao['Probability'].describe())